In [1]:
from helper import load_dotenv

In [2]:
from pathlib import Path
import sqlite3
import re
from tqdm import tqdm
from random import choice

In [3]:
dotenv = load_dotenv()

In [4]:
dotenv['DB_PATH']

PosixPath('/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/10_db/moire_testdata.db')

# Datenbank aktualisieren

# Daten umbenennen

In [5]:
relevant_files = list(dotenv['DATA_DIR'].glob('./*/ps2400dpi150lpi/*.4c.*.*'))

In [6]:
len(relevant_files)

11457

In [8]:
0 / 0

ZeroDivisionError: division by zero

In [9]:
for f in tqdm(relevant_files):
    parts = re.match(r'(.+)\.4c\.(.+)',f.name.replace( f.suffix, '' )).groups()
    new_filename = ".".join(list(parts)) + f.suffix
    out_path = f.parent / new_filename
    check_paths = list(f.parent.parent.glob(f'./vps2400dpi*lpi/{ out_path.name.replace(out_path.suffix,"").replace("4c_600","4c_300") }*'))
  
    f.rename(out_path)

    if len(check_paths) == 0:
        print(f)

  1%|▏                                    | 58/11457 [00:00<00:21, 519.19it/s]

/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/508890/ps2400dpi150lpi/HBZP_1_M_290.p1.4c.Y.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/508890/ps2400dpi150lpi/HBZP_1_M_278.p1.4c.M.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/508890/ps2400dpi150lpi/HBZP_1_M_290.p1.4c.M.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/508890/ps2400dpi150lpi/HBZP_1_M_282.p1.4c.C.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/508890/ps2400dpi150lpi/HBZP_1_M_278.p1.4c.Y.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/508890/ps2400dpi150lpi/HBZP_1_M_274.p1.4c.M.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/508890/ps2400dpi150lpi/HBZP_1_M_274.p1.4c.Y.tif

  8%|██▉                                 | 950/11457 [00:02<00:15, 698.72it/s]

/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613845/ps2400dpi150lpi/NL_pit_text_08_09_EDG__23_ET_96_G_DE_LANGUAGEDOC_nl_NL.p2.4c.Y.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613845/ps2400dpi150lpi/NL_pit_text_94_95_EDG__23_ET_96_G_DE_LANGUAGEDOC_nl_NL.p2.4c.C.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613845/ps2400dpi150lpi/NL_pit_text_86_87_EDG__23_ET_96_G_DE_LANGUAGEDOC_nl_NL.p1.4c.Y.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613845/ps2400dpi150lpi/NL_pit_text_78_79_EDG__23_ET_96_G_DE_LANGUAGEDOC_nl_NL.p2.4c.M.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613845/ps2400dpi150lpi/NL_pit_text_10_11_EDG__23_ET_96_G_DE_LANGUAGEDOC_nl_NL.p1.4c.Y.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/M

  9%|███▎                               | 1088/11457 [00:02<00:16, 623.94it/s]

...

/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0007_T_A_41L.p1.4c.C.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0003_T_NL_41L.p1.4c.C.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0008_T_D_Suffix_90_41L.p1.4c.C.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0064_T_CH_41L.p1.4c.Y.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0093_T_F_41L.p1.4c.Y.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0018_T_F_41L.p1.4c.C.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-Bertel

 99%|█████████████████████████████████▊| 11397/11457 [00:39<00:00, 198.81it/s]

/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0057_T_D_41L.p1.4c.Y.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0033_T_WCH_41L.p1.4c.C.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0067_T_CH_41L.p1.4c.C.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0025_T_WCH_41L.p1.4c.C.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0108_T_WCH_41L.p1 2.4c.M.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0028_T_F_41L.p1.4c.Y.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-Bertelsma

100%|██████████████████████████████████| 11457/11457 [00:39<00:00, 290.40it/s]

/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0025_T_CH_41L.p1.4c.Y.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0102_T_CH_41L.p1 2.4c.C.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0042_T_NL_41L.p1.4c.C.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0064_T_D_Suffix_90_41L.p1.4c.Y.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0111_T_A_41L.p1.4c.Y.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/613010/ps2400dpi150lpi/FC_FS23_S_ES0095_T_A_41L.p1.4c.Y.tif
/Users/frederic.birwe/Library/CloudStorage/OneDrive-Ber